In [1]:
from DataReader import DataReaderChallangePolimi2017
import implicit
from CB_asy import CB_asymmetric_cosine
from CF_IB_asy import CF_IB_asymmetric_cosine
from CF_UB_asy import CF_UB_asymmetric_cosine
from implicit.nearest_neighbours import all_pairs_knn
from sklearn.preprocessing import normalize
from sklearn.utils.sparsefuncs import (inplace_row_scale,inplace_column_scale)
import numpy as np
import scipy.sparse as sp
import pickle

evaluation = True
rebuild = False
dr = DataReaderChallangePolimi2017(evaluation=evaluation,rebuild=rebuild)

urm = dr.getURM_csr()
t_p = dr.target_playlists
t_t = dr.target_tracks    

Date Reader Challange Polimi 2017
Loading data from ./DumpData/DataReaderTrainTestDump
Dataset train-test loaded


In [10]:
#track-album
icm = dr.getICM_csr(albums=True,artists=False,tags=False)
icm = dr.bm25_row(icm)
s = dr.knn_slow1(icm,k=55)
s.setdiag(0)
r_al = urm*s.T
r_al = normalize(r_al, axis=0)
if evaluation: print dr.evaluateMAP(r_al,verbose=False)
r_al = dr.reduceRM(r_al)

0
25000
50000
75000
0.0712221111111


In [9]:
#track-artist
icm = dr.getICM_csr(albums=False,artists=True,tags=False)
icm = dr.bm25_row(icm)
s = dr.knn_slow1(icm,k=55)
s.setdiag(0)
r_ar = urm*s.T
r_ar = normalize(r_ar, axis=0)
if evaluation: print dr.evaluateMAP(r_ar,verbose=False)
r_ar = dr.reduceRM(r_ar)

0
25000
50000
75000
0.0705229722222


In [22]:
#album and artist
icm = dr.getICM_csr(albums=True,artists=True,tags=False)
icm = normalize(icm,axis=0)
icm = dr.bm25_row(icm)
s = dr.knn_slow1(icm,k=55)
r_al_ar = urm*s.T
r_al_ar = normalize(r_al_ar, axis=0)
if evaluation: print dr.evaluateMAP(r_al_ar,verbose=False)
r_al_ar = dr.reduceRM(r_al_ar)

0
25000
50000
75000
0.0793113888889


In [23]:
#tags
icm = dr.getICM_csr(albums=False,artists=False,tags=True)
icm = dr.bm25_row(icm)
s = dr.knn_slow2(icm,k=45)
s.setdiag(0)
r_tag = urm*s.T
r_tag = normalize(r_tag, axis=0)
if evaluation: print dr.evaluateMAP(r_tag, verbose = False)
r_tag = dr.reduceRM(r_tag)

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
0.0390162777778


In [12]:
#cf ub normal
urm_c = urm.copy()
urm_c = dr.bm25_row(urm_c)
s = dr.knn_slow1(urm_c,k=60)
r_cfub = s.T * urm
r_cfub = normalize(r_cfub, axis=1)
if evaluation: print dr.evaluateMAP(r_cfub, verbose = False)
r_cfub = dr.reduceRM(r_cfub)

0
25000
50000
0.0660521666667


In [13]:
#cf ib normal
urm_c = urm.copy()
urm_c = dr.bm25_col(urm_c)
s = dr.knn_slow1(urm_c.T,k=100)
r_cfib = urm * s.T
r_cfib = normalize(r_cfib, axis=0)
if evaluation: print dr.evaluateMAP(r_cfib, verbose = False)
r_cfib = dr.reduceRM(r_cfib)

0
25000
50000
75000
0.0729281388889


In [15]:
#owners
ucm = dr.getOwnersUCM_csr()
ucm = normalize(ucm,axis=0)
ucm = dr.bm25_row(ucm)
s = dr.knn_slow1(ucm,k=65)
r_o = s*urm
r_o = normalize(r_o, axis=0)
if evaluation: print dr.evaluateMAP(r_o,verbose=False)
r_o = dr.reduceRM(r_o)

0
25000
50000
0.0224808888889


In [44]:
############################### Formula best score

In [24]:
r_tot = 2.25 * r_cfib + 1.82 * r_cfub + 1.02*r_al_ar + 0.19* r_tag + 0.75 * r_o + 0.15*r_al + 0.12*r_ar
if evaluation: print dr.evaluateMAP(r_tot,verbose = False)

0.109143722222


In [ ]:
############################## Build submission file

In [16]:
rec = dr.getAllRecommendations(r_tot)
if not evaluation: dr.buildSubmissionFile(recommendation=rec, file_name="a natale puoi boostarlo anche tuuuu 18") ####<------name

User 0 of 10000
User 2500 of 10000
User 5000 of 10000
User 7500 of 10000
Recommendations done
Creating submission file in: ./Submissions/a natale puoi boostarlo anche tuuuu 18
Done


In [ ]:
############################# other things not used in the solution

In [ ]:
#tuning
for p in np.arange(0.5,1.5,0.02):
    r_totx = 1.809 * 1.250 * r_cfib + 1.820 * r_cfub + 1*r_al_ar + 0.19* r_tag + p * r_o + 0.150*r_al + 0.119*r_ar
    score = dr.evaluateMAP(r_totx,verbose = False)
    print ("k = {:1.2f} \t\tmap = {:1.5f}").format(p,score)

In [10]:
#sslim
if evaluation:
    filesslim = "./DumpData/sim_bpr_10e_100k2.npz" # or "./DumpData/sim_bpr_10e_100k.npz"
else:
    filesslim = "./DumpData/sim_bpr_10e_100k_fulltrain.npz"
r_bpr = dr.reduceRM(urm * dr.load_sparse_csr(filesslim))
if evaluation: print dr.evaluateMAP(r_bpr, verbose=False)

In [236]:
#playtlist-album
icm_al = dr.getUCM_csr(albums=True,artists=False)
icm_al = dr.bm25_row(icm_al)
s = all_pairs_knn(icm_al,K=100)
s.setdiag(0)
r_pal = s*urm
r_pal = normalize(r_pal, axis=0)
if evaluation: print dr.evaluateMAP(r_pal,verbose=False)
r_pal = dr.reduceRM(r_pal)

0.0685035277778


In [235]:
#playlist-artist
icm_ar = dr.getUCM_csr(albums=False,artists=True)
icm_ar = dr.bm25_row(icm_ar)
s = all_pairs_knn(icm_ar,K=100)
r_par = s.T*urm
r_par = normalize(r_par, axis=0)
if evaluation: print dr.evaluateMAP(r_par,verbose=False)
r_par = dr.reduceRM(r_par)

0.0607006666667


In [205]:
#cf ub plus
urm_c = urm.copy()
urm_c = dr.bm25_row(urm_c)
urm_c2 = dr.bm25_row(urm_c)
s = all_pairs_knn(urm_c2,K=60)
r_cfubp = s.T * urm_c2
r_cfubp = normalize(r_cfubp, axis=1)
if evaluation: print dr.evaluateMAP(r_cfubp, verbose = False)
r_cfubp = dr.reduceRM(r_cfubp)

0.0713711944444


In [196]:
#cf ib plus
urm_c = urm.copy()
urm_c = dr.bm25_col(urm_c)
urm_c2 = dr.bm25_col(urm_c)
s = all_pairs_knn(urm_c.T,K=100)
#s.setdiag(0)
r_cfibp = urm_c2 * s.T
r_cfibp = normalize(r_cfibp, axis=0)
if evaluation: print dr.evaluateMAP(r_cfibp, verbose = False)
r_cfibp = dr.reduceRM(r_cfibp)

0.0751395277778
